# **NyxRunner: Deployment Otomatis Proyek GitHub ke Google Colab**

Notebook ini membantu menjalankan proyek GitHub secara otomatis di Google Colab dengan langkah-langkah terstruktur.
Setiap bagian memiliki heading, deskripsi, dan log yang jelas agar mudah dipahami, bahkan oleh pemula.

In [1]:
import time
import os

## **1. Clone Repository GitHub**

Bagian ini melakukan cloning repository dari GitHub ke lingkungan Google Colab. Pastikan URL repository sudah benar.

In [2]:
print("[INFO] Memulai proses deployment NyxRunner...")
print("[DEBUG] Working directory saat ini: " + os.getcwd() + "\n")

start_time = time.time()
%cd /content/
print("[DEBUG] Pindah ke direktory: /content/")

[INFO] Memulai proses deployment NyxRunner...
[DEBUG] Working directory saat ini: /content

/content
[DEBUG] Pindah ke direktory: /content/


In [3]:
result = !git clone https://github.com/RozhakDev/RecurCrawl.git 2>&1
if "already exists" in str(result):
    print("[WARNING] Repository sudah ada. Menggunakan versi yang existing.")
    print("[DEBUG] Output git clone: " + str(result))
elif "fatal:" in str(result):
    print("[ERROR] Gagal melakukan clone repository!")
    print("[DEBUG] Error details: " + str(result))
else:
    print("[INFO] Repository berhasil di-clone")
    print("[DEBUG] Output git clone: " + str(result[-1]))

[INFO] Repository berhasil di-clone
[DEBUG] Output git clone: Resolving deltas: 100% (2/2), done.


In [4]:
%cd /content/RecurCrawl
print("[DEBUG] Pindah ke directory project: /content/...")
print("[INFO] Proses clone selesai. Waktu eksekusi: {:.2f} detik".format(time.time() - start_time))

/content/RecurCrawl
[DEBUG] Pindah ke directory project: /content/...
[INFO] Proses clone selesai. Waktu eksekusi: 1.06 detik


## **2. Install Dependencies**

Menginstal semua dependensi yang dibutuhkan proyek dari file `requirements.txt`, dan juga pyngrok untuk membuat tunnel.

In [5]:
print("[INFO] Melakukan instalasi dependensi project...")

if os.path.exists("requirements.txt"):
    print("[INFO] File requirements.txt ditemukan. Memulai instalasi...")
    start_time = time.time()

    result = !pip install -r requirements.txt 2>&1
    if "Successfully installed" in str(result) or "Requirement already satisfied" in str(result):
        print("[INFO] Dependensi utama berhasil diinstal")
        print("[DEBUG] Jumlah package yang diperiksa: {}".format(len([line for line in result if "already satisfied" in line or "Successfully" in line])))
    else:
        print("[WARNING] Mungkin ada masalah dengan instalasi dependensi")
        print("[DEBUG] Output pip install: " + str(result[-3:]))

    print("[INFO] Waktu instalasi requirements: {:.2f} detik".format(time.time() - start_time))
else:
    print("[WARNING] File requirements.txt tidak ditemukan!")

[INFO] Melakukan instalasi dependensi project...
[INFO] File requirements.txt ditemukan. Memulai instalasi...
[INFO] Dependensi utama berhasil diinstal
[DEBUG] Jumlah package yang diperiksa: 25
[INFO] Waktu instalasi requirements: 11.85 detik


In [6]:
print("[INFO] Menginstal pyngrok untuk tunneling...")
start_time = time.time()
result = !pip install pyngrok 2>&1
if "Successfully installed" in str(result):
    print("[INFO] pyngrok berhasil diinstal")
else:
    print("[DEBUG] Status pyngrok: " + str(result[-1]))
print("[INFO] Waktu instalasi pyngrok: {:.2f} detik".format(time.time() - start_time))

[INFO] Menginstal pyngrok untuk tunneling...
[INFO] pyngrok berhasil diinstal
[INFO] Waktu instalasi pyngrok: 9.15 detik


## **3. Konfigurasi Ngrok**

Menambahkan authtoken ngrok dan memastikan tidak ada proses ngrok/flask yang berjalan sebelumnya.

In [7]:
print("[DEBUG] Memeriksa proses ngrok yang sedang berjalan...")
ngrok_processes = !ps aux | grep ngrok | grep -v grep || true
if ngrok_processes:
    print("[INFO] Menghentikan proses ngrok yang existing...")
    !pkill -f ngrok
    print("[DEBUG] Proses ngrok yang dihentikan: {}".format(len(ngrok_processes)))

print("[DEBUG] Memeriksa proses Flask yang sedang berjalan...")
flask_processes = !ps aux | grep flask | grep -v grep || true
if flask_processes:
    print("[INFO] Menghentikan proses Flask yang existing...")
    !pkill -f flask
    print("[DEBUG] Proses Flask yang dihentikan: {}".format(len(flask_processes)))

[DEBUG] Memeriksa proses ngrok yang sedang berjalan...
[DEBUG] Memeriksa proses Flask yang sedang berjalan...


In [8]:
print("[INFO] Menambahkan authtoken ngrok...")
result = !ngrok config add-authtoken  2>&1
if "saved to" in str(result) or "already installed" in str(result):
    print("[INFO] Authtoken ngrok berhasil dikonfigurasi")
else:
    print("[WARNING] Mungkin ada issue dengan konfigurasi authtoken")
    print("[DEBUG] Output ngrok config: " + str(result))

print("[INFO] Konfigurasi ngrok selesai")

[INFO] Menambahkan authtoken ngrok...
[INFO] Authtoken ngrok berhasil dikonfigurasi
[INFO] Konfigurasi ngrok selesai


## **4. Jalankan Server Flask**

Membuka tunnel menggunakan pyngrok di port 5000 dan menampilkan URL publik, kemudian menjalankan server Flask.

In [9]:
print("[INFO] Memulai persiapan server Flask...")

if not os.path.exists("run.py"):
    print("[ERROR] File run.py tidak ditemukan!")
    print("[DEBUG] File yang ada di directory: " + str(os.listdir('.')))
else:
    print("[INFO] File run.py ditemukan. Membuka tunnel...")

    from pyngrok import ngrok
    print("[INFO] Membuka tunnel ngrok di port 5000...")

    try:
        public_url = ngrok.connect(5000)
        print("[SUCCESS] Tunnel berhasil dibuat!")
        print("[INFO] Public URL: {}".format(public_url))
        print("[DEBUG] Tunnel info: {}".format(public_url))

        print("[INFO] Menjalankan server Flask...")
        print("[DEBUG] Memulai proses: python run.py")
        print("-" * 60)
        print("[SYSTEM] Server sedang berjalan. Tekan Ctrl+C untuk menghentikan.")
        print("-" * 60)

        !python run.py
    except Exception as e:
        print("[ERROR] Gagal membuat tunnel: {}".format(str(e)))
        print("[DEBUG] Exception details: {}".format(type(e).__name__))

print("[INFO] Proses deployment selesai. Terima kasih!")

[INFO] Memulai persiapan server Flask...
[INFO] File run.py ditemukan. Membuka tunnel...
[INFO] Membuka tunnel ngrok di port 5000...
[SUCCESS] Tunnel berhasil dibuat!
[INFO] Public URL: NgrokTunnel: "https://b4c84d0250bf.ngrok-free.app" -> "http://localhost:5000"
[DEBUG] Tunnel info: NgrokTunnel: "https://b4c84d0250bf.ngrok-free.app" -> "http://localhost:5000"
[INFO] Menjalankan server Flask...
[DEBUG] Memulai proses: python run.py
------------------------------------------------------------
[SYSTEM] Server sedang berjalan. Tekan Ctrl+C untuk menghentikan.
------------------------------------------------------------
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
 * Debugger is active!
 * Debugger PIN: 245-171-737
127.0.0.1 - - [21/Sep/2025 08:05:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2025 08:05:03] "GET /static/favicon.ico HTTP/1.1" 200 -
[INFO] Proses deployment selesai. Terima kasih